# Recherche Predictive - Vision 2026 (XGBoost Optimise via GridSearch)

Ce notebook presente la pipeline de modelisation basee exclusivement sur **XGBoost Regressor**, avec une phase de recherche par grille (**GridSearchCV**) pour maximiser la precision des prévisions.

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import xgboost as xgb
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
import joblib
import os

In [ ]:
# Chargement et preparation
df_adm = pd.read_csv('../data/raw/admissions_hopital_pitie_2024.csv')
df_adm['date_entree'] = pd.to_datetime(df_adm['date_entree'])
daily_data = df_adm.groupby('date_entree').size().rename('admissions').reset_index()
daily_data = daily_data.set_index('date_entree').asfreq('D', fill_value=0)

def create_features(df):
    df = df.copy()
    df['month_sin'] = np.sin(2 * np.pi * df.index.month / 12)
    df['month_cos'] = np.cos(2 * np.pi * df.index.month / 12)
    df['day_sin'] = np.sin(2 * np.pi * df.index.dayofweek / 7)
    df['day_cos'] = np.cos(2 * np.pi * df.index.dayofweek / 7)
    df['dayofyear'] = df.index.dayofyear
    df['weekofyear'] = df.index.isocalendar().week.astype(int)
    df['lag1'] = df['admissions'].shift(1)
    df['lag7'] = df['admissions'].shift(7)
    df['lag14'] = df['admissions'].shift(14)
    df['roll_mean_7'] = df['admissions'].shift(1).rolling(window=7).mean()
    df['roll_std_7'] = df['admissions'].shift(1).rolling(window=7).std()
    return df.dropna()

features_df = create_features(daily_data)
FEATURES = ['month_sin', 'month_cos', 'day_sin', 'day_cos', 'dayofyear', 
            'weekofyear', 'lag1', 'lag7', 'lag14', 'roll_mean_7', 'roll_std_7']
TARGET = 'admissions'

train = features_df.iloc[:-30]
test = features_df.iloc[-30:]
X_train, y_train = train[FEATURES], train[TARGET]
X_test, y_test = test[FEATURES], test[TARGET]

## Recherche par Grille (GridSearch)
Nous cherchons les hyperparametres optimaux pour l'unique modele XGBoost.

In [ ]:
param_grid = {
    'n_estimators': [500, 1000],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 5, 7]
}

tscv = TimeSeriesSplit(n_splits=3)
grid = GridSearchCV(
    xgb.XGBRegressor(objective='reg:squarederror'), 
    param_grid, 
    cv=tscv, 
    scoring='neg_mean_absolute_error',
    verbose=1
)
grid.fit(X_train, y_train)

print(f"Meilleurs Reglages : {grid.best_params_}")
best_xgb = grid.best_estimator_

## Evaluation Finale

In [ ]:
y_pred = best_xgb.predict(X_test)
print(f"MAE Test Set : {mean_absolute_error(y_test, y_pred):.2f}")

fig = go.Figure()
fig.add_trace(go.Scatter(x=test.index, y=y_test, name='Reel (Dec)', line=dict(color='#005ba1')))
fig.add_trace(go.Scatter(x=test.index, y=y_pred, name='XGBoost Optimise', line=dict(color='#c8102e', dash='dash')))
fig.update_layout(title='Prediction XGBoost avec GridSearch', template='plotly_dark')
fig.show()